In [0]:
# les librairies
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark

In [0]:
# importation des données depuis bucket S3
path = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"
df = spark.read.format("json").option("header", "true").load(path)

# vue schema
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
df.show(5)

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows



In [0]:
# vue 1° enregistrement
df.take(1)

Out[5]: [Row(data=Row(appid=10, categories=['Multi-player', 'Valve Anti-Cheat enabled', 'Online PvP', 'Shared/Split Screen PvP', 'PvP'], ccu=13990, developer='Valve', discount='0', genre='Action', header_image='https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513', initialprice='999', languages='English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean', name='Counter-Strike', negative=5199, owners='10,000,000 .. 20,000,000', platforms=Row(linux=True, mac=True, windows=True), positive=201215, price='999', publisher='Valve', release_date='2000/11/1', required_age='0', short_description="Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.", tags=Row(1980s=266, 1990's=1

In [0]:
#différentes informations
total=df.count()

<span style="color:red">55_691 enregistrements.</span></BR>

In [0]:
# quels sont les 5 meilleurs éditeurs calssés par ordre décroissant ?

df.groupBy("data.publisher").count().orderBy(desc("count")).show(5)

+--------------+-----+
|     publisher|count|
+--------------+-----+
|Big Fish Games|  422|
|        8floor|  202|
|          SEGA|  165|
|Strategy First|  151|
|   Square Enix|  141|
+--------------+-----+
only showing top 5 rows



<span style="color:red">Big Fish Games est largement en tête avec 422 titres publiés.</span></BR>
<span style="color:red">A partir de la 3° place, coude à coude entre éditeurs.</span></BR>

## Les jeux les mieux notés

In [0]:
# Filtre avec un nombre de votes > 100.000 pour avoir des valeurs significatives et ne pas avoir de outliers.

df_jeux = df.select('data.name', 'data.publisher', 'data.positive', 'data.negative')\
    .withColumn('total_reviews', col('positive') + col('negative'))\
    .withColumn('positive_ratio', col('positive') / col('total_reviews'))\
    .filter(col('total_reviews') >= 100000)\
    .orderBy('positive_ratio', ascending=False)\
    .limit(10)

display(df_jeux)

name,publisher,positive,negative,total_reviews,positive_ratio
People Playground,Studio Minus,142920,1649,144569,0.9885936819096763
Portal 2,Valve,305671,3770,309441,0.9878167405094994
Vampire Survivors,poncle,130311,1624,131935,0.9876909084018646
Hades,Supergiant Games,199960,2829,202789,0.9860495391761881
Portal,Valve,111786,1752,113538,0.9845690429635893
RimWorld,Ludeon Studios,142201,2550,144751,0.9823835413917693
Stardew Valley,ConcernedApe,497558,9283,506841,0.9816845914201889
Wallpaper Engine,Wallpaper Engine Team,561096,11031,572127,0.9807193158162436
Helltaker,vanripper,105952,2213,108165,0.9795405168030324
Slay the Spire,Mega Crit Games,116347,2494,118841,0.9790139766578875


In [0]:
# regroupé par date de sortie
df.groupBy("data.release_date").count().orderBy(desc("count")).show(5)

+------------+-----+
|release_date|count|
+------------+-----+
|            |   99|
|  2020/01/17|   74|
|  2022/09/30|   64|
|  2020/10/15|   63|
|  2021/09/30|   62|
+------------+-----+
only showing top 5 rows



In [0]:
# Conversion au format date
release_date = df.select("data.release_date")
release_date = release_date.withColumn("release_date", col("release_date").cast(DateType()))
release_date.printSchema()

root
 |-- release_date: date (nullable = true)



## Gestion des dates

In [0]:
# Eclatement de la date en année, mois, jour
release_explode = df.withColumn("release_date", split(df["data.release_date"], "/"))
release_explode.show(5)

+--------------------+-------+--------------+
|                data|     id|  release_date|
+--------------------+-------+--------------+
|{10, [Multi-playe...|     10| [2000, 11, 1]|
|{1000000, [Single...|1000000|[2021, 05, 14]|
|{1000010, [Single...|1000010|[2020, 10, 16]|
|{1000030, [Multi-...|1000030|[2020, 10, 14]|
|{1000040, [Single...|1000040|[2019, 03, 30]|
+--------------------+-------+--------------+
only showing top 5 rows



In [0]:
# on ne garde que les années
df_date_explode = release_explode.withColumn("release_date", explode("release_date"))
df_date_explode.show(5)

+--------------------+-------+------------+
|                data|     id|release_date|
+--------------------+-------+------------+
|{10, [Multi-playe...|     10|        2000|
|{10, [Multi-playe...|     10|          11|
|{10, [Multi-playe...|     10|           1|
|{1000000, [Single...|1000000|        2021|
|{1000000, [Single...|1000000|          05|
+--------------------+-------+------------+
only showing top 5 rows



In [0]:
# Nombre d'enregistrements par année
counts_release_date = df_date_explode.groupBy("release_date").count().orderBy(desc("count"))
counts_release_date.show(50)

+------------+-----+
|release_date|count|
+------------+-----+
|        2021| 8823|
|        2020| 8305|
|        2018| 7678|
|        2022| 7455|
|        2019| 6968|
|        2017| 6017|
|        2016| 4185|
|        2015| 2576|
|        2014| 1557|
|        2013|  471|
|        2012|  345|
|        2009|  311|
|        2010|  288|
|        2011|  267|
|        2008|  159|
|        2007|   98|
|        2006|   61|
|        2005|    6|
|        2004|    6|
|        2001|    4|
|        1999|    3|
|        2003|    3|
|        2000|    2|
|        1997|    2|
|        2002|    1|
|        1998|    1|
+------------+-----+



<span style="color:red">Steam a réellement décollé à partir de 2006 avec 61 titres jusqu'à 2014 où les publications ont quadruplé.</span></BR>
<span style="color:red">Puis nouvelle expansion à partir de 2016.</span></BR>
<span style="color:red">Il n'y a pas eu d'effet "Covid", ni en +, ni en -.</span></BR>

## Distribution des prix de vente.

In [0]:
# Récupération des données de la colonne price
price = df.select("data.price")
price = price.withColumn("price", col("price").cast(IntegerType()))
price.printSchema()

root
 |-- price: integer (nullable = true)



In [0]:
# Statistiques
price.describe().show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|            55691|
|   mean|773.2849832109317|
| stddev| 1093.13458272345|
|    min|                0|
|    max|            99900|
+-------+-----------------+



<span style="color:red">Grosse disparité de prix, entre gratuité et 99_900$.</span></BR>

In [0]:
# prix le + élevé
max_price = price.select(max("price"))
max_price.show()

+----------+
|max(price)|
+----------+
|     99900|
+----------+



In [0]:
df.select("data.discount").describe().show()

+-------+------------------+
|summary|          discount|
+-------+------------------+
|  count|             55691|
|   mean| 2.603777989262179|
| stddev|12.887080174743176|
|    min|                 0|
|    max|                90|
+-------+------------------+



In [0]:
# Conversion de la colonne discount en entier
discount = df.select("data.discount")
discount = discount.withColumn("discount", col("discount").cast(IntegerType()))
discount.printSchema()

root
 |-- discount: integer (nullable = true)



In [0]:
df_discount = df.select(col('data.discount').alias('discount'))\
    .filter(col('discount') > 0)\
    .count()

print(f'Nombre de jeux en promo : {df_discount}')
print(f'Nombre de jeux total : {total}')
print(f'Ratio promo/total : {df_discount/total*100:.2f}%')

Nombre de jeux en promo : 2518
Nombre de jeux total : 55691
Ratio promo/total : 4.52%


<span style="color:red">Près de 96% des jeux n'ont pas de promotion.</span></BR>

## Queles sont les langues les + représentées sur Steam ?

In [0]:
df.groupBy("data.languages").count().orderBy(desc("count")).show(5, False)

+------------------------------------------+-----+
|languages                                 |count|
+------------------------------------------+-----+
|English                                   |29163|
|English, Russian                          |1881 |
|English, Simplified Chinese               |1200 |
|English, Japanese                         |1015 |
|English, Not supported, Simplified Chinese|683  |
+------------------------------------------+-----+
only showing top 5 rows



In [0]:
# Explosion de la colonne lanquage 
language_explode = df.withColumn("language", split(df["data.languages"], ","))
language_explode.show(5)

+--------------------+-------+--------------------+
|                data|     id|            language|
+--------------------+-------+--------------------+
|{10, [Multi-playe...|     10|[English,  French...|
|{1000000, [Single...|1000000|[English,  Korean...|
|{1000010, [Single...|1000010|[Simplified Chine...|
|{1000030, [Multi-...|1000030|           [English]|
|{1000040, [Single...|1000040|[Simplified Chinese]|
+--------------------+-------+--------------------+
only showing top 5 rows



In [0]:
# EXplosion de la colonne language 
df_lang_explode = language_explode.withColumn("language", explode("language"))
df_lang_explode.printSchema()
df_lang_explode.show()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
# Classement des 10° langues
df_lang_explode.groupBy("language").count().orderBy(desc("count")).show(10)

+--------------------+-----+
|            language|count|
+--------------------+-----+
|             English|54646|
|              German|13996|
|              French|13406|
|             Russian|12839|
|     Spanish - Spain|12224|
|  Simplified Chinese|12213|
|            Japanese|10170|
|             Italian| 9297|
| Portuguese - Brazil| 6739|
|              Korean| 6575|
+--------------------+-----+
only showing top 10 rows



<span style="color:red">L'anglais est de loin la langue la + représentée.</span></BR>

## Les jeux pour les + de 16 ans

In [0]:
required_age = df.select("data.required_age")
required_age = required_age.withColumn("required_age", col("required_age").cast(IntegerType()))
required_age.printSchema()

root
 |-- required_age: integer (nullable = true)



In [0]:
# Statistiques
required_age.describe()

Out[36]: DataFrame[summary: string, required_age: string]

In [0]:
df_required_age = df.filter(col('data.required_age').cast('int') >= 16)\
    .groupBy('data.required_age')\
    .count()\
    .sort('count', ascending=False)\
    .limit(3)

required_age_sum = df_required_age.agg({'count': 'sum'}).collect()[0][0]

print(f'{required_age_sum} jeux interdits au moins de 16 ans, soit {required_age_sum / total * 100:.2f} % des jeux de la plateforme')

299 jeux interdits au moins de 16 ans, soit 0.54 % des jeux de la plateforme


<span style="color:red">299 jeux soit 99.46% des jeux n'ont pas de restictions pour les enfants de moins de 16 ans.</span></BR>

## Analyse sur les genres des jeux.

In [0]:
# Explosion de la colonnes genre.
genre_explode = df.withColumn("genre", split(df["data.genre"], ","))
df_genre_explode = genre_explode.withColumn("genre", explode("genre"))
df_genre_explode.printSchema()
df_genre_explode.show()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

## Classement par genre de jeux

In [0]:
df_genre_explode.groupBy("genre").count().orderBy(desc("count")).show(10)

+-------------+-----+
|        genre|count|
+-------------+-----+
|        Indie|34271|
|       Action|23616|
|       Casual|12312|
|    Adventure|11265|
|    Adventure|10166|
|     Strategy|10045|
|       Casual| 9774|
|   Simulation| 9629|
|          RPG| 8719|
| Early Access| 6125|
+-------------+-----+
only showing top 10 rows



## Les catégories préférées des éditeurs

In [0]:
genre_publisher = df_genre_explode.withColumn("publisher", df.data.publisher)
genre_publisher.groupBy("publisher", "genre").count().sort(desc("count")).show(50)

+--------------------+---------+-----+
|           publisher|    genre|count|
+--------------------+---------+-----+
|      Big Fish Games|Adventure|  391|
|      Big Fish Games|   Casual|  389|
|              8floor|   Casual|  194|
|     Choice of Games|      RPG|  136|
|     Choice of Games|    Indie|  112|
|        Laush Studio|    Indie|  106|
|     Choice of Games|Adventure|   99|
|Alawar Entertainment|Adventure|   94|
|                    |    Indie|   94|
|Alawar Entertainment|   Casual|   94|
|    Devolver Digital|    Indie|   81|
|                SEGA|   Action|   80|
|        Hosted Games|      RPG|   79|
|        Laush Studio|   Casual|   79|
|  Boogygames Studios|    Indie|   78|
|        Kagura Games|      RPG|   77|
|       Sekai Project|    Indie|   76|
|      Piece Of Voxel|    Indie|   76|
|Alawar Entertainment|    Indie|   75|
|         Square Enix|   Action|   75|
|        Hosted Games|    Indie|   74|
|       Artifex Mundi|Adventure|   72|
|  Boogygames Studios| St

## Classement par plateformes Windows/Mac/Linux

In [0]:
# Linux
df.groupBy("data.platforms.linux").count().orderBy(desc("count")).show()

+-----+-----+
|linux|count|
+-----+-----+
|false|47233|
| true| 8458|
+-----+-----+



In [0]:
df.groupBy("data.platforms.mac").count().orderBy(desc("count")).show()

+-----+-----+
|  mac|count|
+-----+-----+
|false|42921|
| true|12770|
+-----+-----+



In [0]:
df.groupBy("data.platforms.windows").count().orderBy(desc("count")).show()

+-------+-----+
|windows|count|
+-------+-----+
|   true|55676|
|  false|   15|
+-------+-----+



In [0]:
print("Percentage of games available in Linux:", 8458/ df.count())
print("Percentage of games available in Mac:", 12770/ df.count())
print("Percentage of games available in Windows:", 55676/ df.count())

Percentage of games available in Linux: 0.15187373184176978
Percentage of games available in Mac: 0.22930096424916055
Percentage of games available in Windows: 0.9997306566590652


<span style="color:red">% Linux 15.18%.</span></BR>
<span style="color:red">% Mac 22.93%.</span></BR>
<span style="color:red">% Windows 99.97%.</span></BR>

## Influence de la plateforme sur le genre de jeux

In [0]:
# linux
genre_explode_linux = df_genre_explode.withColumn("linux", df.data.platforms.linux,)
genre_explode_linux = genre_explode_linux.filter(genre_explode_linux["linux"] == True)
genre_explode_linux.groupBy("genre").count().orderBy(desc("count")).show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|      Indie| 5912|
|     Action| 3369|
|  Adventure| 1945|
|     Casual| 1838|
|   Strategy| 1707|
|     Casual| 1467|
|        RPG| 1438|
| Simulation| 1399|
|  Adventure| 1357|
|      Indie| 1066|
+-----------+-----+
only showing top 10 rows



In [0]:
# mac
genre_explode_mac = df_genre_explode.withColumn("mac", df.data.platforms.mac,)
genre_mac = genre_explode_mac.filter(genre_explode_mac["mac"] == True)
genre_mac.groupBy("genre").count().orderBy(desc("count")).show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|      Indie| 8367|
|     Action| 4549|
|  Adventure| 3150|
|     Casual| 2776|
|   Strategy| 2769|
|     Casual| 2354|
| Simulation| 2191|
|        RPG| 2117|
|  Adventure| 1889|
|      Indie| 1568|
+-----------+-----+
only showing top 10 rows



In [0]:
# windows
genre_explode_windows = df_genre_explode.withColumn("windows", df.data.platforms.windows,)
genre_explode_windows = genre_explode_windows.filter(genre_explode_windows["windows"] == True)
genre_explode_windows.groupBy("genre").count().orderBy(desc("count")).show(10)

+-------------+-----+
|        genre|count|
+-------------+-----+
|        Indie|34267|
|       Action|23612|
|       Casual|12309|
|    Adventure|11262|
|    Adventure|10165|
|     Strategy|10042|
|       Casual| 9773|
|   Simulation| 9626|
|          RPG| 8718|
| Early Access| 6125|
+-------------+-----+
only showing top 10 rows

